In [ ]:
import requests
import pandas as pd

from pyhive import presto
from requests.auth import HTTPBasicAuth
from contextlib import closing

In [ ]:
def get_trino_connection(user, password, host='enterprisehadoopuser.seagate.com', port=8081):
    requests.packages.urllib3.disable_warnings()
    req_kw = {'auth': HTTPBasicAuth(user, password),'verify':False}

    conn = presto.connect(
        host=host,
        port=int(port), 
        protocol='https',
        catalog='hive',
        username=user,
        requests_kwargs=req_kw
    )
    return conn

In [ ]:
ehc_conn = get_trino_connection('kor_knime_SVC', 'p7aryp5sZ9')

In [ ]:
# Sample DataFrame
df = pd.read_csv('./data/diabetes 5.csv')

# Define Table Name
table_name = "oig_gid_769083"

In [ ]:
cols_txt = ' DECIMAL,\n'.join([f'COL{x:02}' for x in range(1, len(df.columns)+1)])
cols_txt += ' DECIMAL'

create_query = f'''
CREATE TABLE edputils.{table_name} (
{cols_txt}
)
'''

columns = [f'COL{x:02}' for x in range(1, len(df.columns)+1)]

insert_sql = f'''
INSERT INTO edputils.{table_name} ({','.join(columns)}) 
VALUES 
    
'''

val_txt = ',\n'.join([str(tuple(x)) for x in df.values])
insert_sql = f'''
INSERT INTO edputils.{table_name} ({', '.join(columns)}) 
VALUES 
{val_txt}
'''

In [ ]:
with closing(ehc_conn.cursor()) as cur:
    cur.execute("DROP TABLE IF EXISTS edputils.oig_gid_769083")
    cur.fetchall()
    cur.execute(create_query)
    cur.fetchall()
    cur.execute(insert_sql)
    cur.fetchall()
    cur.execute("set session hive.compression_codec='SNAPPY'")
    cur.fetchall()
    cur.execute("set session hive.collect_column_statistics_on_write=false")
    cur.fetchall()
    cur.execute("CREATE TABLE edputils.oig_gid_769083_pq with (format = 'PARQUET') as SELECT * FROM edputils.oig_gid_769083")
    cur.fetchall()